# 1. ollama exaone 설치

In [ ]:
!pip install colab-xterm

!pip install langchain langgraph langchain-community chromadb sqlite-utils
curl -fsSL https://ollama.com/install.sh | sh

올라마 서버 실행

In [ ]:
ollama serve & ollama pull exaone3.5:2.4b

올라마 종료

In [ ]:
!ps aux | grep ollama
# ollama   1360934  0.4  0.6 11169072 209564 ?     Ssl  11:33   0:57 /usr/local/bin/ollama serve

# !sudo systemctl stop ollama

# Mysql 접속

라이브러리 설치

In [ ]:
!pip install mysql-connector-python

Mysql 사용자 정보 입력

In [2]:
import mysql.connector

# MySQL 접속 정보 설정
# !!! 이 부분을 실제 사용자 정보로 변경해야 합니다. !!!
DB_CONFIG = {
    "host": "localhost",   # MySQL 서버 주소 (로컬인 경우 'localhost')
    "user": "admin", # MySQL 사용자 이름
    "password": "1qazZAQ!", # MySQL 비밀번호
    "database": "final" # 사용할 데이터베이스 이름 (미리 생성되어 있어야 함)
}

Mysql 접속 확인

In [3]:
def check_mysql_connection():
    """MySQL 데이터베이스 연결을 시도하고 결과를 출력하는 함수"""
    print(f"[{DB_CONFIG['user']}@{DB_CONFIG['host']}] 로 MySQL 연결을 시도합니다...")
    
    try:
        # DB 연결 시도
        conn = mysql.connector.connect(
            host=DB_CONFIG["host"],
            user=DB_CONFIG["user"],
            password=DB_CONFIG["password"],
            # database=DB_CONFIG["database"] # 특정 DB 없이 서버 접속만 확인할 경우 주석 처리
        )
        
        # 연결 성공
        print("\n🎉 **MySQL 접속 성공!**")
        print(f"서버 버전: {conn.get_server_info()}")
        
        # 연결 종료
        conn.close()

    except mysql.connector.Error as err:
        # 연결 실패 시 오류 처리
        print("\n❌ **MySQL 접속 실패!**")
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("오류: 사용자 이름 또는 비밀번호가 잘못되었습니다. (Access Denied)")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print(f"오류: 데이터베이스 '{DB_CONFIG['database']}'가 존재하지 않습니다.")
        else:
            print(f"알 수 없는 오류 발생: {err}")
    
# 함수 실행
check_mysql_connection()

[admin@localhost] 로 MySQL 연결을 시도합니다...

🎉 **MySQL 접속 성공!**
서버 버전: 8.0.42-0ubuntu0.20.04.1


/tmp/ipykernel_2604912/326273039.py:16: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  print(f"서버 버전: {conn.get_server_info()}")


패키지 설치

In [ ]:
pip install pymysql langchain-community langchain-text-splitters langchain-core

In [4]:
import pymysql
from langchain_core.documents import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:

# MySQL 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

def build_rag_chroma():
    # MySQL의 documents 테이블에서 데이터를 로드하여 Chroma 벡터스토어를 생성하고 저장합니다.
    conn = None
    documents = []

    try:
        # MySQL 연결
        conn = pymysql.connect(**DB_CONFIG)
        print("✅ MySQL 연결 성공")

        with conn.cursor() as cursor:
            # documents 테이블에서 summary와 id 조회
            cursor.execute("SELECT summary, id FROM documents")
            rows = cursor.fetchall()

            if not rows:
                print("⚠️ 로드된 데이터가 없습니다. 'documents' 테이블을 확인해주세요.")
                return

            # Document 객체로 변환
            for row in rows:
                summary_text = row[0]
                doc_id = row[1]
                doc = Document(
                    page_content=summary_text,
                    metadata={"source": "mysql", "table": "documents", "id": doc_id}
                )
                documents.append(doc)

            print(f"✅ MySQL에서 {len(documents)}개 문서 로드 완료")

            # 로드된 문서 미리보기 출력
            for i, doc in enumerate(documents):
                print(f"\n--- 문서 #{i + 1} (ID: {doc.metadata.get('id', 'N/A')}) ---")
                print(f"  Metadata: {doc.metadata}")
                print(f"  Content (일부): {doc.page_content[:200]}...")

    except pymysql.Error as err:
        print(f"❌ MySQL 오류: {err}")
        return
    finally:
        if conn:
            conn.close()
            print("🔒 MySQL 연결 해제")

    # 텍스트 분할 (청킹)
    splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    split_docs = splitter.split_documents(documents)
    print(f"\n✅ 청킹 완료. 총 {len(split_docs)}개 청크 생성")

    # 임베딩 모델 설정
    try:
        embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
        print("✅ 임베딩 모델 설정 완료")
    except Exception as e:
        print(f"❌ 임베딩 모델 설정 오류: {e}")
        print("   Ollama 서버가 실행 중인지 확인하세요.")
        return

    # Chroma 벡터스토어 생성 및 저장
    print("\n⏳ 벡터스토어 생성 및 임베딩 중...")
    rag_path = "./rag_chroma/documents/summary/"
    
    db = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=rag_path
    )
    db.persist()
    
    print(f"🎉 RAG Chroma 벡터스토어 구축 완료!")
    print(f"   저장 경로: {rag_path}")

# 실행
build_rag_chroma()

✅ MySQL 연결 성공
✅ MySQL에서 15개 문서 로드 완료

--- 문서 #1 (ID: 1) ---
  Metadata: {'source': 'mysql', 'table': 'documents', 'id': 1}
  Content (일부): 카카오의 홍민택 최고제품책임자(CPO)가 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 개편 배경과 의도를 설명했다. 홍 CPO는 기존 메시징 서비스의 성장 정체와 함께 사용자 피드백 부족을 인지하고 개편을 진행했다고 밝혔다. 주요 변경 사항으로는 친구탭 피드 노출 조정과 기존 메시지 기능 유지로의 복귀가 포함되며, 트래픽은 유지되지만 사용자 불편...

--- 문서 #2 (ID: 2) ---
  Metadata: {'source': 'mysql', 'table': 'documents', 'id': 2}
  Content (일부): ...

--- 문서 #3 (ID: 3) ---
  Metadata: {'source': 'mysql', 'table': 'documents', 'id': 3}
  Content (일부): 대전 국가정보자원관리원에서 발생한 화재(9월 29일 발생)는 무정전·전원 장치 리튬이온 배터리 이전 작업 중 폭발로 인해 발생한 것으로 추정된다. 경찰은 당시 현장 책임자와 작업자 등 4명을 업무상 실화 혐의로 입건하고, CCTV 영상 및 작업 관련 자료 분석과 함께 전문가 조사를 진행 중이며, 국가 시스템 안정화를 위해 강제 수사 가능성도 검토 중이다. ...

--- 문서 #4 (ID: 4) ---
  Metadata: {'source': 'mysql', 'table': 'documents', 'id': 4}
  Content (일부): 지난달 20일, 주차장에서 주차 자리를 맡은 여성이 차주와 충돌한 후 특수폭행 혐의로 고소당했다. 차주 A씨는 parking 자리에서 아주머니에게 '비켜 주세요'라고 말했지만, 아주머니가 먼저 차를 움직여 접촉이 발생했다. A씨는 초기에는 충격을 받지 못했

/tmp/ipykernel_1800095/2110142050.py:68: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")


🎉 RAG Chroma 벡터스토어 구축 완료!
   저장 경로: ./rag_chroma/documents/summary/


/tmp/ipykernel_1800095/2110142050.py:84: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


## 1. 벡터스토어 검색 함수

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

def search_vectorstore(query, top_k=3):
    """
    Chroma 벡터스토어에서 유사도 검색을 수행합니다.
    
    Args:
        query: 검색할 질문/키워드
        top_k: 반환할 상위 결과 개수 (기본값: 3)
    """
    # 벡터스토어 경로
    rag_path = "./rag_chroma/documents/summary/"
    
    try:
        # 임베딩 모델 로드
        embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
        
        # 기존 벡터스토어 로드
        db = Chroma(
            persist_directory=rag_path,
            embedding_function=embeddings
        )
        
        print(f"✅ 벡터스토어 로드 완료: {rag_path}")
        print(f"🔍 검색 쿼리: '{query}'\n")
        
        # 유사도 검색 (similarity_search)
        results = db.similarity_search(query, k=top_k)
        
        # 결과 출력
        print(f"📊 검색 결과 (상위 {len(results)}개):\n")
        for i, doc in enumerate(results):
            print(f"--- 결과 #{i + 1} ---")
            print(f"문서 ID: {doc.metadata.get('id', 'N/A')}")
            print(f"출처: {doc.metadata.get('source', 'N/A')}")
            print(f"내용: {doc.page_content[:300]}...")
            print()
        
        return results
        
    except Exception as e:
        print(f"❌ 검색 오류: {e}")
        return []

# 검색 테스트
search_vectorstore("카카오톡 개편")

/tmp/ipykernel_2604912/161535806.py:17: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
/tmp/ipykernel_2604912/161535806.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


✅ 벡터스토어 로드 완료: ./rag_chroma/documents/summary/
🔍 검색 쿼리: '카카오톡 개편'

📊 검색 결과 (상위 3개):

--- 결과 #1 ---
문서 ID: 15
출처: mysql
내용: 카카오의 최고제품책임자(CPO) 홍민택이 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 기존 메시지 중심 서비스에서 체류형 서비스로의 확장 필요성을 인정하고 현재의 트래픽 유지와 사용자 불편 최소화에 초점을 맞추겠다고 밝혔다. 개편 과정에서의 소통 부족에 대해 사과한 가운데, 친구탭 피드 노출 문제와 기존 콘텐츠의 개선 방안을 제시하며 정식 서비스 도입 가능성을 언급했다....

--- 결과 #2 ---
문서 ID: 1
출처: mysql
내용: 카카오의 홍민택 최고제품책임자(CPO)가 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 개편 배경과 의도를 설명했다. 홍 CPO는 기존 메시징 서비스의 성장 정체와 함께 사용자 피드백 부족을 인지하고 개편을 진행했다고 밝혔다. 주요 변경 사항으로는 친구탭 피드 노출 조정과 기존 메시지 기능 유지로의 복귀가 포함되며, 트래픽은 유지되지만 사용자 불편 최소화에 초점을 두겠다고 강조했다. 이에 대한 책임을 다짐하며 이용자의 의견 수렴을 약속했다....

--- 결과 #3 ---
문서 ID: 14
출처: mysql
내용: 서울시가 한강버스의 해상 시운전 결과를 알고 있었음에도 불구하고, 정식 운항을 앞두고 실제 속도 미달 사실을 숨기고 홍보를 진행하며 시민 기만 논란이 일었다. 8월부터 실시된 시운전에서 한강버스의 평균 속도는 목표치인 17노트(시속 31.5km)를 크게 밑돌았고, 서울시는 이를 공개하지 않고 시간 단축을 강조했으나 실제로는 시간이 더 소요되었다. 이러한 과정에서 한강버스 운항 중단, 기계 결함 등 안전 문제가 발생했으며, 서울시는 선박 품질 문제와 관련해 법적 조치 가능성까지 제기되고 있다....



[Document(metadata={'table': 'documents', 'source': 'mysql', 'id': 15}, page_content='카카오의 최고제품책임자(CPO) 홍민택이 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 기존 메시지 중심 서비스에서 체류형 서비스로의 확장 필요성을 인정하고 현재의 트래픽 유지와 사용자 불편 최소화에 초점을 맞추겠다고 밝혔다. 개편 과정에서의 소통 부족에 대해 사과한 가운데, 친구탭 피드 노출 문제와 기존 콘텐츠의 개선 방안을 제시하며 정식 서비스 도입 가능성을 언급했다.'),
 Document(metadata={'source': 'mysql', 'table': 'documents', 'id': 1}, page_content='카카오의 홍민택 최고제품책임자(CPO)가 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 개편 배경과 의도를 설명했다. 홍 CPO는 기존 메시징 서비스의 성장 정체와 함께 사용자 피드백 부족을 인지하고 개편을 진행했다고 밝혔다. 주요 변경 사항으로는 친구탭 피드 노출 조정과 기존 메시지 기능 유지로의 복귀가 포함되며, 트래픽은 유지되지만 사용자 불편 최소화에 초점을 두겠다고 강조했다. 이에 대한 책임을 다짐하며 이용자의 의견 수렴을 약속했다.'),
 Document(metadata={'source': 'mysql', 'id': 14, 'table': 'documents'}, page_content='서울시가 한강버스의 해상 시운전 결과를 알고 있었음에도 불구하고, 정식 운항을 앞두고 실제 속도 미달 사실을 숨기고 홍보를 진행하며 시민 기만 논란이 일었다. 8월부터 실시된 시운전에서 한강버스의 평균 속도는 목표치인 17노트(시속 31.5km)를 크게 밑돌았고, 서울시는 이를 공개하지 않고 시간 단축을 강조했으나 실제로는 시간이 더 소요되었다. 이러한 과정에서 한강버스 운항 중단, 기계 결함 등 안전 문제가 발생했으며, 서울시는 선박 품질 문제와 관련해 법적 조치 가능성까지

## 2. 유사도 점수와 함께 검색

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

def search_vectorstore(query, top_k=3):
    """
    Chroma 벡터스토어에서 검색 (유사도 100% 기준으로 표시)
    최대/최소 점수를 기준으로 정규화
    """
    rag_path = "./rag_chroma/documents/summary/"
    
    try:
        embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
        db = Chroma(
            persist_directory=rag_path,
            embedding_function=embeddings
        )
        
        print(f"✅ 벡터스토어 로드 완료")
        print(f"🔍 검색 쿼리: '{query}'\n")
        
        # 유사도 점수와 함께 검색
        results = db.similarity_search_with_score(query, k=top_k)
        
        if not results:
            print("검색 결과가 없습니다.")
            return []
        
        # 점수 범위 추출
        scores = [score for _, score in results]
        min_score = min(scores)
        max_score = max(scores)
        
        print(f"📊 검색 결과 (상위 {len(results)}개):\n")
        for i, (doc, score) in enumerate(results):
            # 최대/최소값으로 정규화 (0~100%)
            # 거리가 작을수록(min에 가까울수록) 100%에 가까움
            if max_score == min_score:
                similarity_percent = 100.0
            else:
                similarity_percent = (1 - (score - min_score) / (max_score - min_score)) * 100
            
            print(f"--- 결과 #{i + 1} (유사도: {similarity_percent:.2f}%) ---")
            print(f"문서 ID: {doc.metadata.get('id', 'N/A')}")
            print(f"출처: {doc.metadata.get('source', 'N/A')}")
            print(f"내용: {doc.page_content}")
            print(f"원본 거리 점수: {score:.4f}")
            print()
        
        return results
        
    except Exception as e:
        print(f"❌ 검색 오류: {e}")
        return []

# 검색 테스트
search_vectorstore("카카오톡 개편", top_k=8)

✅ 벡터스토어 로드 완료
🔍 검색 쿼리: '카카오톡 개편'

📊 검색 결과 (상위 8개):

--- 결과 #1 (유사도: 100.00%) ---
문서 ID: 15
출처: mysql
내용: 카카오의 최고제품책임자(CPO) 홍민택이 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 기존 메시지 중심 서비스에서 체류형 서비스로의 확장 필요성을 인정하고 현재의 트래픽 유지와 사용자 불편 최소화에 초점을 맞추겠다고 밝혔다. 개편 과정에서의 소통 부족에 대해 사과한 가운데, 친구탭 피드 노출 문제와 기존 콘텐츠의 개선 방안을 제시하며 정식 서비스 도입 가능성을 언급했다.
원본 거리 점수: 12770.4990

--- 결과 #2 (유사도: 80.43%) ---
문서 ID: 1
출처: mysql
내용: 카카오의 홍민택 최고제품책임자(CPO)가 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 개편 배경과 의도를 설명했다. 홍 CPO는 기존 메시징 서비스의 성장 정체와 함께 사용자 피드백 부족을 인지하고 개편을 진행했다고 밝혔다. 주요 변경 사항으로는 친구탭 피드 노출 조정과 기존 메시지 기능 유지로의 복귀가 포함되며, 트래픽은 유지되지만 사용자 불편 최소화에 초점을 두겠다고 강조했다. 이에 대한 책임을 다짐하며 이용자의 의견 수렴을 약속했다.
원본 거리 점수: 13855.5586

--- 결과 #3 (유사도: 21.53%) ---
문서 ID: 14
출처: mysql
내용: 서울시가 한강버스의 해상 시운전 결과를 알고 있었음에도 불구하고, 정식 운항을 앞두고 실제 속도 미달 사실을 숨기고 홍보를 진행하며 시민 기만 논란이 일었다. 8월부터 실시된 시운전에서 한강버스의 평균 속도는 목표치인 17노트(시속 31.5km)를 크게 밑돌았고, 서울시는 이를 공개하지 않고 시간 단축을 강조했으나 실제로는 시간이 더 소요되었다. 이러한 과정에서 한강버스 운항 중단, 기계 결함 등 안전 문제가 발생했으며, 서울시는 선박 품질 문제와 관련해 법적 조치 가능성까지 제기되고 있다.
원

[(Document(metadata={'table': 'documents', 'source': 'mysql', 'id': 15}, page_content='카카오의 최고제품책임자(CPO) 홍민택이 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 기존 메시지 중심 서비스에서 체류형 서비스로의 확장 필요성을 인정하고 현재의 트래픽 유지와 사용자 불편 최소화에 초점을 맞추겠다고 밝혔다. 개편 과정에서의 소통 부족에 대해 사과한 가운데, 친구탭 피드 노출 문제와 기존 콘텐츠의 개선 방안을 제시하며 정식 서비스 도입 가능성을 언급했다.'),
  12770.4990234375),
 (Document(metadata={'id': 1, 'source': 'mysql', 'table': 'documents'}, page_content='카카오의 홍민택 최고제품책임자(CPO)가 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 개편 배경과 의도를 설명했다. 홍 CPO는 기존 메시징 서비스의 성장 정체와 함께 사용자 피드백 부족을 인지하고 개편을 진행했다고 밝혔다. 주요 변경 사항으로는 친구탭 피드 노출 조정과 기존 메시지 기능 유지로의 복귀가 포함되며, 트래픽은 유지되지만 사용자 불편 최소화에 초점을 두겠다고 강조했다. 이에 대한 책임을 다짐하며 이용자의 의견 수렴을 약속했다.'),
  13855.55859375),
 (Document(metadata={'id': 14, 'source': 'mysql', 'table': 'documents'}, page_content='서울시가 한강버스의 해상 시운전 결과를 알고 있었음에도 불구하고, 정식 운항을 앞두고 실제 속도 미달 사실을 숨기고 홍보를 진행하며 시민 기만 논란이 일었다. 8월부터 실시된 시운전에서 한강버스의 평균 속도는 목표치인 17노트(시속 31.5km)를 크게 밑돌았고, 서울시는 이를 공개하지 않고 시간 단축을 강조했으나 실제로는 시간이 더 소요되었다. 이러한 과정에서 한강버스 운항 중단, 기계 결함 등 안

# 유사도 실행 예시

In [43]:
# search_with_score("카카오톡", top_k=3)
search_vectorstore("전유성 별세", top_k=5)

✅ 벡터스토어 로드 완료
🔍 검색 쿼리: '전유성 별세'

📊 검색 결과 (상위 5개):

--- 결과 #1 (유사도: 100.00%) ---
문서 ID: 7
출처: mysql
내용: 76세의 전유성, 한국 코미디계의 거장이 폐기흉 악화로 갑작스럽게 세상을 떠났다. 지난 몇 달간 병원 입원과 퇴원을 반복하다 2025년 9월 25일 서울아산병원에서 별동대장으로 장례를 치를 예정이다. 그의 유일한 딸이 곁을 지키며, 코미디계는 큰 충격을 받았다. 전유성은 '살아있는 전설'로 불리며 후배들에게 큰 영감을 줬던 인물이다.
원본 거리 점수: 14429.8379

--- 결과 #2 (유사도: 11.95%) ---
문서 ID: 14
출처: mysql
내용: 서울시가 한강버스의 해상 시운전 결과를 알고 있었음에도 불구하고, 정식 운항을 앞두고 실제 속도 미달 사실을 숨기고 홍보를 진행하며 시민 기만 논란이 일었다. 8월부터 실시된 시운전에서 한강버스의 평균 속도는 목표치인 17노트(시속 31.5km)를 크게 밑돌았고, 서울시는 이를 공개하지 않고 시간 단축을 강조했으나 실제로는 시간이 더 소요되었다. 이러한 과정에서 한강버스 운항 중단, 기계 결함 등 안전 문제가 발생했으며, 서울시는 선박 품질 문제와 관련해 법적 조치 가능성까지 제기되고 있다.
원본 거리 점수: 17332.5938

--- 결과 #3 (유사도: 11.55%) ---
문서 ID: 11
출처: mysql
내용: 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 부모의 압박 속에서도 지식 추구를 이어갔으나, 깊은 외로움과 사회적 고립감을 겪으며 결국 노숙 생활을 선택했다. 그는 저렴한 생활비로도 의미 있는 삶을 살고 있으며, 어린 시절의 상처를 치유하고 진정한 열정을 찾기 위해 노력 중이다. 이는 과도한 책임 요구와 사회적 압박 속에서의 방황과 실패를 보여주는 사례로 주목받고 있다.
원본 거리 점수: 17345.5508

--- 결과 #4 (유사도: 2.60%) ---
문서 ID: 15
출처: mysql
내용

[(Document(metadata={'source': 'mysql', 'table': 'documents', 'id': 7}, page_content="76세의 전유성, 한국 코미디계의 거장이 폐기흉 악화로 갑작스럽게 세상을 떠났다. 지난 몇 달간 병원 입원과 퇴원을 반복하다 2025년 9월 25일 서울아산병원에서 별동대장으로 장례를 치를 예정이다. 그의 유일한 딸이 곁을 지키며, 코미디계는 큰 충격을 받았다. 전유성은 '살아있는 전설'로 불리며 후배들에게 큰 영감을 줬던 인물이다."),
  14429.837890625),
 (Document(metadata={'id': 14, 'source': 'mysql', 'table': 'documents'}, page_content='서울시가 한강버스의 해상 시운전 결과를 알고 있었음에도 불구하고, 정식 운항을 앞두고 실제 속도 미달 사실을 숨기고 홍보를 진행하며 시민 기만 논란이 일었다. 8월부터 실시된 시운전에서 한강버스의 평균 속도는 목표치인 17노트(시속 31.5km)를 크게 밑돌았고, 서울시는 이를 공개하지 않고 시간 단축을 강조했으나 실제로는 시간이 더 소요되었다. 이러한 과정에서 한강버스 운항 중단, 기계 결함 등 안전 문제가 발생했으며, 서울시는 선박 품질 문제와 관련해 법적 조치 가능성까지 제기되고 있다.'),
  17332.59375),
 (Document(metadata={'id': 11, 'source': 'mysql', 'table': 'documents'}, page_content='중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 부모의 압박 속에서도 지식 추구를 이어갔으나, 깊은 외로움과 사회적 고립감을 겪으며 결국 노숙 생활을 선택했다. 그는 저렴한 생활비로도 의미 있는 삶을 살고 있으며, 어린 시절의 상처를 치유하고 진정한 열정을 찾기 위해 노력 중이다. 이는 과도한 책임 요구와 사회적 압박 속에서의 방황과 실패를 보여주는 사례로 주목받고 있다.'),
  17345.55078125)

# DB 리스트 불러오기

In [27]:
import pymysql # MySQL 접속 라이브러리 임포트

DB_CONFIG = {
    'host': 'localhost', 
    'user': 'admin', 
    'password': '1qazZAQ!', 
    'db': 'final',
    'charset': 'utf8mb4'
}

def query_documents_db_mysql():
    conn = None # 연결 객체 초기화
    
    try:
        # 1. MySQL 서버에 연결 (DB_CONFIG 사용)
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor() # 커서 생성

        # 2. 데이터 조회
        cursor.execute("SELECT * FROM documents")
        documents = cursor.fetchall() # 모든 결과 가져오기

        # 3. 데이터 출력
        print("\n---documents Table Data (MySQL)---")
        for row in documents:
            print(f"id: {row[0]}, title: {row[1]}, file_location: {row[4]}...")

    except pymysql.Error as err:
        print(f"❌ MySQL 작업 중 오류 발생: {err}")
        
    finally:
        # 4. 연결 해제
        if conn:
            conn.close()

# 함수 실행
query_documents_db_mysql()


---documents Table Data (MySQL)---
id: 1, title: 카카오톡 개편 논란 해명, 홍민택 CPO의 입장 표명, file_location: /home/alpaco/final_project/file_search/fileList/IT기사.docx...
id: 2, title: 문화체육관광부, 저소득층 대상 '문화생활권 지원 사업' 안내, file_location: fileList/2025년 신문 구독 지원 신청 전 필독사항.pdf...
id: 3, title: 대전 국가정보자원관리원 화재 원인 조사, 경찰 업무상실화 혐의로 작업자 등 4명 입건, file_location: fileList/사회기사정보.docx...
id: 4, title: 주차 자리 막은 여성 고소 사건, file_location: fileList/파렴치한.docx...
id: 5, title: 이재명 대통령, 국군의날 기념행사 개최, file_location: fileList/국민의군대.docx...
id: 6, title: 이재명 정부 청년정책 추진 방향, file_location: fileList/소름돋는정책원해.docx...
id: 7, title: 전유성, 한국 코미디계의 전설이 세상을 떠남, file_location: fileList/전유성별세.docx...
id: 8, title: 아틀레티코 마드리드의 유망주 메이슨 그린우드 영입 협상, file_location: fileList/해외축구_기사.docx...
id: 9, title: **한국 9월 수출 역대 최대 달성**, file_location: fileList/경제기사.docx...
id: 10, title: 윤석열 전 대통령 교정 시절 직원 24시간 수발 폭로, file_location: fileList/랭킹뉴스.docx...
id: 11, title: 중국 엘리트 청년의 노숙 생활 고백, file_location: fileList/세계기사.docx...
id: 12, title: 이재명 대통령,

추가 패키지 설치

In [ ]:
!pip install grandalf

In [ ]:
from typing import TypedDict, List
from langchain import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
import pymysql

# 상태 정의
class AgentState(TypedDict):
    query: str
    keywords: str
    file_paths: list  # 변경: 파일 정보 저장
    relevant_docs: str
    result: str

# DB 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

llm = Ollama(model="exaone3.5:2.4b")

# 키워드 추출 에이전트
keyword_prompt = PromptTemplate.from_template(
    """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
    벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
    \n질문: {query}"""
)

def extractor_agent(state: AgentState):
    chain = LLMChain(llm=llm, prompt=keyword_prompt)
    keywords = chain.run({"query": state["query"]})
    return {**state, "keywords": keywords.strip()}

# RAG + 파일 정보 추출 에이전트
def rag_agent(state: AgentState):
    embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
    vectorstore = Chroma(
        persist_directory="./rag_chroma/documents/summary/", 
        embedding_function=embeddings
    )
    
    # 벡터스토어 검색 (점수 포함)
    results = vectorstore.similarity_search_with_score(state["keywords"], k=10)
    
    # 점수 범위 계산
    scores = [score for _, score in results]
    min_score = min(scores)
    max_score = max(scores)
    
    # MySQL 연결 및 파일 정보 조회
    conn = None
    file_entries = []
    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        for doc, score in results:
            # 유사도 계산 (0-100%)
            if max_score == min_score:
                similarity = 100.0
            else:
                similarity = (1 - (score - min_score)/(max_score - min_score)) * 100
            
            # 문서 ID 추출 (metadata에 id가 있다고 가정)
            doc_id = doc.metadata.get('id')
            
            # MySQL에서 파일 정보 조회
            if doc_id:
                cursor.execute("""
                    SELECT file_name, file_location, summary 
                    FROM documents 
                    WHERE id = %s
                """, (doc_id,))
                row = cursor.fetchone()
                if row:
                    file_entries.append({
                        'name': row[0],
                        'location': row[1],
                        'summary': row[2],
                        'relevance': round(similarity, 1)
                    })
    except Exception as e:
        print(f"DB 오류: {e}")
    finally:
        if conn: conn.close()
    
    # 문서 요약 생성
    doc_summary = "\n".join([d.page_content for d, _ in results])
    
    return {**state, "file_paths": file_entries, "relevant_docs": doc_summary}

# 최종 답변 에이전트
answer_prompt = PromptTemplate.from_template(
    """
    --- 제공된 정보 ---
    질문: {query}
    키워드: {keywords}
    문서 요약:
    {relevant_docs}
    파일 경로:
    {file_paths}
    
    ⚠️ 절대 지어내거나 추측하지 마세요.
    제공된 정보 안에 없으면 " 관련 정보가 없습니다"라고만 답변하세요.

    출력 규칙:
    각 파일에 대해 다음 형식으로 정확히 출력하세요:
    # 파일명: [파일명]
    # 파일위치: [파일 전체 경로]
    # 관련성: [소수점 1자리 %]
    # 설명: [문서 요약]

    관련 정보가 없으면 " 관련 정보가 없습니다"
    """
)

def answer_agent(state: AgentState):
    entries = state.get("file_paths", [])
    if not entries:
        return {**state, "result": " 관련 정보가 없습니다"}
    
    answer = ""
    for i, entry in enumerate(entries, 1):
        answer += f"{i}순위\n"
        answer += f"# 파일명: {entry['name']}\n"
        answer += f"# 파일위치: {entry['location']}\n"
        answer += f"# 관련성: {entry['relevance']}%\n"
        answer += f"# 설명: {entry['summary'][:200]}...\n\n"
    
    return {**state, "result": answer.strip()}

# LangGraph 연결
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag", rag_agent)
graph.add_node("answer", answer_agent)
graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag")
graph.add_edge("rag", "answer")
graph.add_edge("answer", END)
app = graph.compile()

# 실행 예시
query = "에이닷비지"
state = {"query": query, "keywords": "", "file_paths": [], "relevant_docs": "", "result": ""}
result = app.invoke(state)
print(result["result"])

1순위
# 파일명: IT기사.docx
# 파일위치: fileList/IT기사.docx
# 관련성: 100.0%
# 설명: 카카오의 최고제품책임자(CPO) 홍민택이 카카오톡 개편 논란에 대해 사내 공지를 통해 사과하며, 기존 메시지 중심 서비스에서 체류형 서비스로의 확장 필요성을 인정하고 현재의 트래픽 유지와 사용자 불편 최소화에 초점을 맞추겠다고 밝혔다. 개편 과정에서의 소통 부족에 대해 사과한 가운데, 친구탭 피드 노출 문제와 기존 콘텐츠의 개선 방안을 제시하며 정식 서비스...

2순위
# 파일명: IT과학정보.docx
# 파일위치: fileList/IT과학정보.docx
# 관련성: 71.0%
# 설명: SK텔레콤이 SK AX와 함께 개발한 업무용 인공지능 에이전트 '에이닷 비즈'가 하반기 내 SK그룹 25개사 구성원들에게 도입되어 회의록 작성 시간을 60%, 보고서 작성 시간을 약 40% 단축시키는 등 업무 효율성을 획기적으로 향상시키고 있다. 이 플랫폼은 정보 검색, 일정 관리, 자연어 기반 업무 지원 등을 제공하며, 특히 에이전트 빌더와 스토어 기능을...

3순위
# 파일명: 세계기사.docx
# 파일위치: fileList/세계기사.docx
# 관련성: 61.4%
# 설명: 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 부모의 압박 속에서도 지식 추구를 이어갔으나, 깊은 외로움과 사회적 고립감을 겪으며 결국 노숙 생활을 선택했다. 그는 저렴한 생활비로도 의미 있는 삶을 살고 있으며, 어린 시절의 상처를 치유하고 진정한 열정을 찾기 위해 노력 중이다. 이는 과도한 책임 요구와 사회적 압박 속에서의 방황과 실패를 보여주는 사...

4순위
# 파일명: 경제기사.docx
# 파일위치: fileList/경제기사.docx
# 관련성: 36.2%
# 설명: 한국의 9월 수출은 작년 동기 대비 12.7% 증가한 659억5천만 달러로 역대 최대치를 기록했다. 반도체와 자동차 수출이 주도적인 역할을 했으며, 특히 반도체는 전년 대비 22.0% 증가한 역대 최대